In [ ]:
from pathlib import Path

model_path = Path("../models/")
dir_list = [path for path in model_path.iterdir() if path.is_dir()]
processed_dir_list = [path for path in dir_list if (path / "input.tif").exists()]

default_root_dir = (
    model_path
    / "20240417_095627_0-0.999999_1000x32x256x256_skip=0_l=10_d=7_sf=32_ds=2at10_f=10.0_z=3_g=8_sd=0_b=tri_a=gelu_b=4_e=500_p=32"
)
ckpt_path = default_root_dir / "version_0/checkpoints/epoch=79-step=20000.ckpt"
input_path = default_root_dir / "input.tif"
output_path = default_root_dir / "inference.tif"
data_dir = Path("../../tempdata/Simulated_data/")
data_path = data_dir / "RES_4341_p=0.06250_m=0.05898.tif"
ground_truth_path = data_dir / "ref.tif"

In [ ]:
# Run a gpu inference
from inference import gpu_patch_inference
from spadgapmodels import SPADGAP
import torch
import numpy as np
from pathlib import Path
from tifffile import imread, imwrite

model = SPADGAP.load_from_checkpoint(ckpt_path)

input = imread(data_path)
output = gpu_patch_inference(
    model,
    input[:512].astype(np.float32),
    min_overlap=32,
    device=1,
)

imwrite(default_root_dir / "inference.tif", output)


In [ ]:
from utils import group_metrics

input = input[:512].astype(float)
image = output[:512]
ground_truth = imread(ground_truth_path)[:512].astype(float)

group_metrics(input, image, ground_truth, default_root_dir, device=torch.device("cuda:0"))

In [ ]:
torch.cuda.get_device_properties(torch.device("cuda:0")).total_memory

In [ ]:
from metrics import ImageMetrics, StackMetrics, StackMetricsGroups
import torch
import seaborn as sns

property_list = ["mse", "psnr", "ssim", "ncc", "brisque", "niqe"]

image_size = 256
frame_count = 16
ref = torch.randn(frame_count, image_size, image_size) * 100
image = ref + torch.randn(frame_count, image_size, image_size) / 2
metrics = StackMetrics(image, ref, property_list=property_list)

ref2 = torch.randn(frame_count, image_size, image_size) * 100
image2 = ref2 + torch.randn(frame_count, image_size, image_size) * 2
metrics2 = StackMetrics(image2, ref2, property_list=property_list)

stack_group = StackMetricsGroups([metrics, metrics2], property_list=property_list)


stack_group.plot_group_trends()
stack_group.plot_group_stats(kind="bar")

In [ ]:
from utils import clean_directories
clean_directories()

In [ ]:
from metrics import StackMetrics, StackMetricsGroups
import torch
from pathlib import Path
from tifffile import imread

length = 512
device = 0

input = imread(input_path)[:length].astype(float)
output = imread(output_path)[:length].astype(float)
ground_truth = imread(ground_truth_path)[:length].astype(float)

for i in range(length):
    input[i] = (input[i] - input[i].min()) / (input[i].max() - input[i].min())
    output[i] = (output[i] - output[i].min()) / (output[i].max() - output[i].min())
    ground_truth[i] = (ground_truth[i] - ground_truth[i].min()) / (
        ground_truth[i].max() - ground_truth[i].min()
    )

metric_list = ["mse", "psnr", "ssim"]
metric1 = StackMetrics(
    output,
    ground_truth,
    metric_list=metric_list,
    device=torch.device(f"cuda:{device}"),
)
metric2 = StackMetrics(
    input,
    ground_truth,
    metric_list=metric_list,
    device=torch.device(f"cuda:{device}"),
)
metric_group = StackMetricsGroups([metric1, metric2], ["processed", "raw"], metric_list)
metric_group.plot_group_stats(
    save=True, save_dir=default_root_dir, save_name="group_stats"
)
metric_group.plot_group_trends(
    save=True, save_dir=default_root_dir, save_name="group_trends"
)
metric1.stats_df.to_csv(default_root_dir / "processed_stats.csv")
metric2.stats_df.to_csv(default_root_dir / "raw_stats.csv")
metric1.values_df.to_csv(default_root_dir / "processed_values.csv")
metric2.values_df.to_csv(default_root_dir / "raw_values.csv")

In [3]:
from pathlib import Path
import yaml
import pandas as pd

meta_list = {}
for model_dir in processed_dir_list:
    meta_dir = model_dir / "metadata.yml"
    if meta_dir.exists():
        with open(meta_dir, "r") as file:
            metadata = yaml.safe_load(file)
        data = metadata.pop("data", {})
        model = metadata.pop("model", {})
        optimizer = model.pop("optimizer", {})
        optimizer_config = model.pop("optimizer_config", {})
        output = {
            "dir": str(meta_dir),
            **data,
            **model,
            **optimizer,
            **optimizer_config,
            **metadata,
        }
        time_stamp = output.pop("time_stamp")
        meta_list[time_stamp] = output
    metric_dir = model_dir / "processed_stats.csv"
    if metric_dir.exists():
        metrics = pd.read_csv(metric_dir)
        metrics_long = pd.melt(
            metrics, id_vars=["Stat"], var_name="metric", value_name="value"
        )
        result_dict = {
            f"{stat}_{metric}": value for stat, metric, value in metrics_long.values
        }
        meta_list[time_stamp] = {**meta_list[time_stamp], **result_dict}

metadf = pd.DataFrame.from_dict(meta_list).T.to_csv(model_path / "metadata_summary.csv")

In [ ]:
from tifffile import imread, imwrite
import numpy as np
from utils import group_metrics
import torch
from tqdm import tqdm
import sys

ground_truth = imread(ground_truth_path)[:512].astype(float)
for dir in tqdm(processed_dir_list, file=sys.stdout):
    input = imread(dir / "input.tif")[:512].astype(float)
    image = imread(dir / "inference.tif")[:512].astype(float)
    out = group_metrics(input, image, ground_truth, dir, device=torch.device("cuda:1"))
    print(f"{dir.stem}: {out}")


In [ ]:
import matplotlib.pyplot as plt

dir = processed_dir_list[3]
ground_truth = imread(ground_truth_path)[:512].astype(float)
input = imread(dir / "input.tif")[:512].astype(float)
image = imread(dir / "inference.tif")[:512].astype(float)
group_metrics(input, image, ground_truth, dir, device=torch.device("cuda:1"))

In [ ]:
from pathlib import Path
from tifffile import imread, imwrite
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

path = Path("../video_models/20240422_223901_Gated_pball_1_1000x32x256x256_skip=0_l=10_d=5_sf=32_ds=2at10_f=10.0_z=3_g=8_sd=0_b=tri_a=gelu_b=4_e=250_p=32/800x500x500_infernece.tif")
data = imread(path)


In [ ]:
data_means = data.mean(axis=(1, 2))
data = data / data_means[:, None, None]
ewma = pd.Series(data_means).ewm(span=400).mean().values
output = data * ewma[:, None, None]
output_path = path.parent / "800x500x500_infernece_normalzied.tif"
imwrite(output_path, output)

In [ ]:
plt.plot(ewma)

In [ ]:
from pathlib import Path
from spadio import SPADFolder, SPADData  # noqa
import matplotlib.pyplot as plt
import numpy as np

data_dir = Path("../../tempdata/")
data_path = data_dir / "acq00005"


try:
    input_folder = SPADFolder(data_path)
except FileNotFoundError:
    raise FileNotFoundError("Input folder does not exist")
input = input_folder.spadstack[:100]
data = input.stack
data_means = np.sum(data, axis=(1,2 ))
plt.plot(data_means)
plt.savefig(data_path / "data_means.png")